In [ ]:
# Block 0: Documentation

print('Script to create a GIF animation of image files\n')
print('Version 1.0, August 26, 2022\n')
print('Written using using Python v3.9\n')
print('Author: Dr. Amy Huff (IMSG at NOAA/NESDIS/STAR), amy.huff@noaa.gov\n')
print('This script allows the user to select one of three options to create a GIF animation from image files; each animation option has different strengths and limitations.\n')
print('Block 1 imports modules and libraries. Blocks 2-4 are functions that require no input from the user; there is no visible output from these blocks. The user specifies animation parameters in Block 5 and obtains output from Block 6.\n')
print('**Please acknowledge the NOAA/NESDIS/STAR Aerosols and Atmospheric Composition Science Team if using any of this code in your work/research!**')

In [ ]:
# Block 1: Import Python packages

# Libraries for creating animations
from PIL import Image
import imageio

# Library to make plots
import matplotlib.pyplot as plt 
import matplotlib.image as mgimg
from matplotlib import animation

# Module to set filesystem paths appropriate for user's operating system
from pathlib import Path

# Modules to create interactive menus in Jupyter Notebook
from IPython.display import display
import ipywidgets as widgets

# Import supporting functions needed to run script
# These functions moved to external .py file to make training Notebook cleaner
import supporting_functions

In [ ]:
# Block 2: Create an animation of multiple graphics files using Imageio
# Imageio is simple and works well for images that contain continuous colorbars (e.g., AOD), but it makes a larger GIF file
# "animation_name", "time_interval": parameter variables from widget menus, set in main function
# "file_list", "file_path": parameter variable assigned in main function

def imageio_animation(file_list, animation_name, file_path, time_interval):

    # Make an empty list to store graphics files
    images = []

    # Loop through graphics files and append to animation
    for file in file_list:
        images.append(imageio.imread(file))

    # Save animation to user-specified directory using user-specified file name
    animation_name = animation_name + '_imageio.gif'
    imageio.mimsave(str(file_path / animation_name), images, 'GIF-PIL', duration=time_interval)

In [ ]:
# Block 3: Create an animation of multiple graphics files using python image library (Pillow)
# Pillow makes a relatively small GIF, but it does not convert continuous colorbars well (e.g. AOD)
# "animation_name", "time_interval": parameter variables from widget menus, set in main function
# "file_list", "file_path": parameter variable assigned in main function

def pillow_animation(file_list, animation_name, file_path, time_interval):

    # Make an empty list to store figures
    frames = []

    # Loop through graphics files and append
    for file in file_list:
        new_frame = Image.open(file)
        frames.append(new_frame)

    # Save animation to user-specified directory using user-specified file name
    pil_duration = time_interval*1000  # Time in milliseconds between frames
    animation_name = animation_name + '_pillow.gif'
    frames[0].save(str(file_path / animation_name), format='GIF', append_images=frames[1:], save_all=True, duration=pil_duration, loop=0)

    # Close the graphics files
    for file in file_list:
        new_frame.close()

In [ ]:
# Block 4: Create an animation of multiple graphics files using Matplotlib
# Matplotlib's animator is slower and has lower resolution compared to Pillow and Imageio, but it makes a smaller GIF file
# "animation_name", "time_interval": parameter variables from widget menus, set in main function
# "file_list", "file_path": parameter variable assigned in main function

def mpl_animation(file_list, animation_name, file_path, time_interval):
    
    # Set up figure and unlabeled axes
    fig = plt.figure(figsize=(18,15), tight_layout=True)
    ax = plt.axes()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Make an empty list to store graphics files
    images = []
    
    # Loop through graphics files, reading each and appending to list
    for file in file_list:
        image = mgimg.imread(file)
        image_plot = plt.imshow(image)
        images.append([image_plot])
    
    # Turn axis off to prevent black border around animation
    ax.axis('off')

    # Create animation
    mpl_animation = animation.ArtistAnimation(fig, images)

    # Save animation to user-specified directory using user-specified file name
    animation_name = animation_name + '_matplotlib.gif'
    writer_gif = animation.PillowWriter(fps=(1/time_interval)) 
    mpl_animation.save(str(file_path / animation_name), writer=writer_gif)    

In [ ]:
# Block 5: Enter settings to create animation file using interactive Jupyter Notebook widgets

# Run this block *once* to generate menus
# When main function is run, it reads "(widget-menu-variable).value" of each menu selection
# Do NOT re-run block if you change menu selections! Re-running block resets menus to defaults)!

# Print warning message
print('If you change menu selections (e.g., to make another animation), do NOT re-run this block!\nRe-running will re-set all menus to their defaults!')

# Create radiobutton menu to select directory where graphics files are located/where animation will be saved
directory_caption = widgets.Label(value='SELECT DIRECTORY WHERE GRAPHICS FILES ARE LOCATED & ANIMATION FILE WILL BE SAVED', layout=widgets.Layout(height='20px'))
radiobutton = widgets.RadioButtons(options=[('Current Working Directory', 1), ('Specify a Directory: (e.g., D://Data)', 2)], disabled=False, layout=widgets.Layout(height='40px'))
directory = widgets.Text(disabled=True, layout=widgets.Layout(width='500px', height='30px'))

# Function to enable text entry only if radiobutton to "specify a directory" is selected
def handle_directory_change(change):
    directory.disabled=False if change.new == 2 else True

# Monitor values of radiobuttons
radiobutton.observe(handle_directory_change, names='value')

# Display directory widgets
display(directory_caption, radiobutton, directory)

# Create text widget to enter name of animation file
animation_name_caption = widgets.Label(value='ENTER NAME FOR SAVED ANIMATION FILE (e.g., abi_aod_animation):', layout=widgets.Layout(height='30px'))
animation_name = widgets.Text(layout=widgets.Layout(width='400px', height='20px'))

# Format animation settings menus to display side-by-side
animation_name_menus = widgets.HBox([animation_name_caption, animation_name])

# Display animation file name widgets
display(animation_name_menus)

# Create drop-down menus using widgets for animation settings
# Formatting settings for drop-down menus
style = {'description_width':'250px'}
layout = widgets.Layout(width='360px')
animation_type = widgets.Dropdown(options=[('Imageio'), ('Pillow'), ('Matplotlib')], description='Animation Generator:', style=style, layout=layout)
time_interval = widgets.Dropdown(options=[('0.5', 0.5), ('1', 1), ('2', 2), ('3', 3), ('4', 4), ('5', 5)], description='Time Interval between Frames (seconds):', style=style, layout=layout)
file_format = widgets.Dropdown(options=[('png'), ('jpg'), ('pdf')], description='Format of Graphics Files to Animate:', style=style, layout=layout)

# Display drop-down menus
display(animation_type, time_interval, file_format)

In [ ]:
# Block 6: Main Function (create animation GIF)
# Selections from widget menus (animation settings) imported using "(widget-menu-variable).value"

# Main function
if __name__ == "__main__":
    
    # Set directory where graphics files are located & animation file will be saved (as pathlib.Path object)
    if radiobutton.value == 1:  # 1=cwd, 2=user-specified directory
        file_path = Path.cwd()  # Set current working directory as pathlib.Path object
    else:
        file_path_error_message = supporting_functions.check_directory(directory.value)  # Check for errors
        if file_path_error_message == 0:
            file_path = Path(directory.value)  # Set user-entered directory as pathlib.Path object
        else:
            file_path = 'error'
    
    # Check user-specified information for errors; if none, proceed to create animation
    # Notify user if errors in user-specified directory
    if file_path == 'error':
        if file_path_error_message == 1:
            print('The directory you entered does not exist. Try again.')
        elif file_path_error_message == 2:
            print('You opted to enter a directory but the field is blank. Try again.')
        elif file_path_error_message == 3:
            print('There is a syntax error in the the entered directory name. Try again.')
    else:
        # Collect all graphics files with user-specified file format in selected directory
        # If no files found, notify user
        file_list = sorted(file_path.glob('*aod*.' + file_format.value))
        if len(file_list) == 0:
            print('No files with the specified graphics file format found in the selected directory. Try again.')
        else:
            # Create animation
            if animation_type.value == 'Imageio':
                imageio_animation(file_list, animation_name.value, file_path, time_interval.value)
            elif animation_type.value == 'Pillow':
                pillow_animation(file_list, animation_name.value, file_path, time_interval.value)
            elif animation_type.value == 'Matplotlib':
                mpl_animation(file_list, animation_name.value, file_path, time_interval.value)

            print('Animation done!')